# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
!pip install requests beautifulsoup4 schedule pytest  --quiet

You should consider upgrading via the 'C:\Users\perfe\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
# Импортирт всех необходимых бибилиотек
import time
import requests
import schedule
from bs4 import BeautifulSoup
import re
import json

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
def get_book_data(book_url: str) -> dict:
    """
    Парсит данные о книге с указанного URL.
    
    Args:
        book_url (str): URL страницы книги для парсинга
        
    Returns:
        dict: Словарь с информацией о книге
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    try:
        response = requests.get(book_url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Название книги
        title = soup.find('h1').text.strip()
        
        # Цена
        price = soup.find('p', class_='price_color').text.strip()
        
        # Рейтинг
        rating_class = soup.find('p', class_='star-rating')['class'][1]
        rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
        rating = rating_map.get(rating_class, 0)
        
        # Количество в наличии
        stock_text = soup.find('p', class_='instock availability').text.strip()
        stock_match = re.search(r'\((\d+) available\)', stock_text)
        stock = int(stock_match.group(1)) if stock_match else 0
        
        # Описание
        description_element = soup.find('div', id='product_description')
        description = ""
        if description_element:
            description_sibling = description_element.find_next_sibling('p')
            if description_sibling:
                description = description_sibling.text.strip()
        
        # Таблица Product Information
        product_info = {}
        table = soup.find('table', class_='table table-striped')
        if table:
            rows = table.find_all('tr')
            for row in rows:
                header = row.find('th').text.strip()
                value = row.find('td').text.strip()
                product_info[header] = value
        
        book_data = {
            'title': title,
            'price': price,
            'rating': rating,
            'stock': stock,
            'description': description,
            'upc': product_info.get('UPC', ''),
            'product_type': product_info.get('Product Type', ''),
            'price_excl_tax': product_info.get('Price (excl. tax)', ''),
            'price_incl_tax': product_info.get('Price (incl. tax)', ''),
            'tax': product_info.get('Tax', ''),
            'availability': product_info.get('Availability', ''),
            'num_reviews': product_info.get('Number of reviews', '')
        }
        
        return book_data
        
    except Exception as e:
        print(f"Ошибка при парсинге {book_url}: {e}")
        return {}
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 3,
 'stock': 22,
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up the

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(is_save: bool = False, max_pages: int = None) -> list:
    """
    Parses books from catalog pages.
    
    Args:
        is_save (bool): If True, saves data to books_data.txt file
        max_pages (int, optional): Maximum number of pages to parse.
                                  If None, parses all pages.
        
    Returns:
        list: List of dictionaries with book information
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    import requests
    from bs4 import BeautifulSoup
    import time
    
    base_url = "http://books.toscrape.com/catalogue/"
    all_books = []
    page = 1
    
    print("Starting parsing...")
    if max_pages:
        print(f"Limit: {max_pages} pages")
    else:
        print("Parsing all available pages")
    
    while True:
        # Проверка лимита страниц
        if max_pages and page > max_pages:
            print(f"Reached limit of {max_pages} pages")
            break
            
        if page == 1:
            url = f"{base_url}page-1.html"
        else:
            url = f"{base_url}page-{page}.html"
        
        print(f"Parsing page {page}: {url}")
        
        try:
            response = requests.get(url, timeout=10)
            
            # Если страницы не существует, заканчивает цикл
            if response.status_code == 404:
                print(f"Reached last page ({page-1} pages total)")
                break
                
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            books = soup.find_all('article', class_='product_pod')
            
            if not books:
                print("No books found on page")
                break
            
            print(f"Found {len(books)} books on page {page}")
            
            book_count = 0
            for book in books:
                # Get link to book page
                book_link = book.find('h3').find('a')['href']
                
                if book_link.startswith('../../../'):
                    full_book_url = f"http://books.toscrape.com/catalogue/{book_link[9:]}"
                else:
                    full_book_url = f"http://books.toscrape.com/catalogue/{book_link}"
                
                book_data = get_book_data(full_book_url)
                if book_data:
                    all_books.append(book_data)
                    book_count += 1
                
                time.sleep(0.1)
            
            print(f"Page {page}: successfully collected {book_count} books")
            
            next_button = soup.find('li', class_='next')
            if not next_button:
                print(f"No more pages to parse (total {page} pages)")
                break
                
            page += 1
            
        except requests.RequestException as e:
            print(f"Network error while parsing page {page}: {e}")
            break
        except Exception as e:
            print(f"Error while parsing page {page}: {e}")
            break
    
    print(f"Parsing completed. Total collected {len(all_books)} books from {page-1} pages")
    
    # Сохранение в файл
    if is_save and all_books:
        try:
            with open('books_data.txt', 'w', encoding='utf-8') as f:
                f.write("BOOKS DATA\n")
                f.write("=" * 80 + "\n")
                f.write(f"Total books: {len(all_books)}\n")
                f.write(f"Total pages: {page-1}\n")
                f.write(f"Collection time: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("=" * 80 + "\n\n")
                
                for i, book in enumerate(all_books, 1):
                    f.write(f"BOOK #{i}\n")
                    f.write(f"Title: {book.get('title', 'N/A')}\n")
                    f.write(f"Price: {book.get('price', 'N/A')}\n")
                    f.write(f"Rating: {book.get('rating', 'N/A')}/5\n")
                    f.write(f"Stock: {book.get('stock', 'N/A')} items\n")
                    f.write(f"Description: {book.get('description', 'N/A')}\n")
                    f.write(f"UPC: {book.get('upc', 'N/A')}\n")
                    f.write(f"Product Type: {book.get('product_type', 'N/A')}\n")
                    f.write(f"Price (excl. tax): {book.get('price_excl_tax', 'N/A')}\n")
                    f.write(f"Price (incl. tax): {book.get('price_incl_tax', 'N/A')}\n")
                    f.write(f"Tax: {book.get('tax', 'N/A')}\n")
                    f.write(f"Availability: {book.get('availability', 'N/A')}\n")
                    f.write(f"Number of reviews: {book.get('num_reviews', 'N/A')}\n")
                    f.write("-" * 80 + "\n\n")
            
            print(f"Full data saved to books_data.txt")
            
        except Exception as e:
            print(f"Error saving file: {e}")
    
    return all_books
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
res = scrape_books(is_save=True, max_pages=None)
print(type(res), len(res))

Starting parsing...
Parsing all available pages
Parsing page 1: http://books.toscrape.com/catalogue/page-1.html
Found 20 books on page 1
Page 1: successfully collected 20 books
Parsing page 2: http://books.toscrape.com/catalogue/page-2.html
Found 20 books on page 2
Page 2: successfully collected 20 books
Parsing page 3: http://books.toscrape.com/catalogue/page-3.html
Found 20 books on page 3
Page 3: successfully collected 20 books
Parsing page 4: http://books.toscrape.com/catalogue/page-4.html
Found 20 books on page 4
Page 4: successfully collected 20 books
Parsing page 5: http://books.toscrape.com/catalogue/page-5.html
Found 20 books on page 5
Page 5: successfully collected 20 books
Parsing page 6: http://books.toscrape.com/catalogue/page-6.html
Found 20 books on page 6
Page 6: successfully collected 20 books
Parsing page 7: http://books.toscrape.com/catalogue/page-7.html
Found 20 books on page 7
Page 7: successfully collected 20 books
Parsing page 8: http://books.toscrape.com/catalog

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [1]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import time
import schedule

def scheduled_scraping():
    """
    Функция для автоматического запуска парсинга по расписанию.
    Будет вызываться только в 19:00 каждый день.
    """
    print(f"\n{'='*60}")
    print(f"АВТОМАТИЧЕСКИЙ ПАРСИНГ: Запуск в {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Здесь будет запуск scrape_books(is_save=True, max_pages=None)")
    print(f"{'='*60}")

# Настройка расписания - ежедневно в 19:00
schedule.every().day.at("19:00").do(scheduled_scraping)

print("✓ ПЛАНИРОВЩИК: Инициализирован")
print("✓ Задача: Ежедневный парсинг в 19:00")
print(f"✓ Текущее время: {time.strftime('%H:%M:%S')}")
print("✓ Планировщик настроен, но НЕ запущен")

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

✓ ПЛАНИРОВЩИК: Инициализирован
✓ Задача: Ежедневный парсинг в 19:00
✓ Текущее время: 15:45:23
✓ Планировщик настроен, но НЕ запущен


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
import os

PROJECT_ROOT = r"C:\Users\perfe\OneDrive\Рабочий стол\Projects\panfilova_tatiana_hw3"

original_dir = os.getcwd()

try:
    os.chdir(PROJECT_ROOT)
    print("Текущая директория:", os.getcwd())
    
    !pytest tests/scraper_test.py -v
finally:
    os.chdir(original_dir)

Текущая директория: C:\Users\perfe\OneDrive\Рабочий стол\Projects\panfilova_tatiana_hw3
============================= test session starts =============================
platform win32 -- Python 3.10.0, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\perfe\AppData\Local\Programs\Python\Python310\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\perfe\OneDrive\Рабочий стол\Projects\panfilova_tatiana_hw3
plugins: anyio-4.11.0
collecting ... collected 3 items

tests/scraper_test.py::test_get_book_data_returns_dict PASSED            [ 33%]
tests/scraper_test.py::test_get_book_data_has_required_keys PASSED       [ 66%]
tests/scraper_test.py::test_scrape_books_returns_books PASSED            [100%]

============================= 3 passed in 15.62s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```